**About Book Crossing Dataset**<br>

This dataset has been compiled by Cai-Nicolas Ziegler in 2004, and it comprises of three tables for users, books and ratings. Explicit ratings are expressed on a scale from 1-10 (higher values denoting higher appreciation) and implicit rating is expressed by 0.

Reference: http://www2.informatik.uni-freiburg.de/~cziegler/BX/ 

**Objective**

This project entails building a Book Recommender System for users based on user-based and item-based collaborative filtering approaches.

#### Execute the below cell to load the datasets

In [1]:
import pandas as pd
import numpy as np
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader, Dataset, SVD, SVDpp, NMF, KNNBaseline, accuracy

In [2]:
#Loading data
books = pd.read_csv("books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']

users = pd.read_csv('users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']

ratings = pd.read_csv('ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\sumans\AppData\Local\Continuum\anaconda3\lib\site-packages\IPytho

### Check no.of records and features given in each dataset

In [3]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
ISBN                 271360 non-null object
bookTitle            271360 non-null object
bookAuthor           271359 non-null object
yearOfPublication    271360 non-null object
publisher            271358 non-null object
imageUrlS            271360 non-null object
imageUrlM            271360 non-null object
imageUrlL            271357 non-null object
dtypes: object(8)
memory usage: 16.6+ MB


In [4]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
userID      278858 non-null int64
Location    278858 non-null object
Age         168096 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
userID        1149780 non-null int64
ISBN          1149780 non-null object
bookRating    1149780 non-null int64
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


## Exploring books dataset

In [6]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


### Drop last three columns containing image URLs which will not be required for analysis

In [7]:
books = books.drop(['imageUrlS','imageUrlM','imageUrlL'],axis=1)

In [8]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


**yearOfPublication**

### Check unique values of yearOfPublication


In [9]:
books['yearOfPublication'].unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

As it can be seen from above that there are some incorrect entries in this field. It looks like Publisher names 'DK Publishing Inc' and 'Gallimard' have been incorrectly loaded as yearOfPublication in dataset due to some errors in csv file.


Also some of the entries are strings and same years have been entered as numbers in some places. We will try to fix these things in the coming questions.

### Check the rows having 'DK Publishing Inc' as yearOfPublication

In [10]:
books[books['yearOfPublication'] == 'DK Publishing Inc'].count()

ISBN                 2
bookTitle            2
bookAuthor           2
yearOfPublication    2
publisher            2
dtype: int64

### Drop the rows having `'DK Publishing Inc'` and `'Gallimard'` as `yearOfPublication`

In [11]:
books = books.loc[books['yearOfPublication']!='DK Publishing Inc']

### Change the datatype of yearOfPublication to 'int'

In [12]:
books = books.loc[books['yearOfPublication']!='Gallimard']

In [13]:
books.dtypes

ISBN                 object
bookTitle            object
bookAuthor           object
yearOfPublication    object
publisher            object
dtype: object

### Drop NaNs in `'publisher'` column


In [14]:
books = books.dropna(subset=['publisher'])

In [15]:
books['publisher'].isna().sum()

0

## Exploring Users dataset

In [16]:
print(users.shape)
users.head(10)

(278858, 3)


,userID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
5,6,"santa monica, california, usa",61.0
6,7,"washington, dc, usa",NaN
7,8,"timmins, ontario, canada",NaN
8,9,"germantown, tennessee, usa",NaN
9,10,"albacete, wisconsin, spain",26.0


### Get all unique values in ascending order for column `Age`

In [17]:
print(sorted(users['Age'].unique(),reverse=False))

[nan, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 113.0, 114.0, 115.0, 116.0, 118.0, 119.0, 123.0, 124.0, 127.0, 128.0, 132.0, 133.0, 136.0, 137.0, 138.0, 140.0, 141.0, 143.0, 146.0, 147.0, 148.0, 151.0, 152.0, 156.0, 157.0, 159.0, 162.0, 168.0, 172.0, 175.0, 183.0, 186.0, 189.0, 199.0, 200.0, 201.0, 204.0, 207.0, 208.0, 209.0, 210.0, 212.0, 219.0, 220.0, 223.0, 226.0

Age column has some invalid entries like nan, 0 and very high values like 100 and above

### Values below 5 and above 90 do not make much sense for our book rating case...hence replace these by NaNs

In [18]:
def replace_age(age,mean):
    
    if age>90:
        return mean
    elif age<5:
        return mean 
    elif np.isnan(age):
        return mean  
    else:
        return age

In [19]:
users['Age'].mean()

34.75143370454978

In [20]:
users['Age'] = users.apply(lambda x: replace_age(x['Age'],users['Age'].mean()),axis=1)

### Replace null values in column `Age` with mean

In [21]:
#Done in the above step under a single function

### Change the datatype of `Age` to `int`

In [22]:
users['Age'] = users['Age'].astype(int)

In [23]:
print(sorted(users.Age.unique()))

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]


## Exploring the Ratings Dataset

### check the shape

In [24]:
ratings.shape

(1149780, 3)

In [25]:
n_users = users.shape[0]
n_books = books.shape[0]

In [26]:
ratings.head(5)

,userID,ISBN,bookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### Ratings dataset should have books only which exist in our books dataset. Drop the remaining rows

In [27]:
ratings = pd.merge(ratings, books, on='ISBN', how='inner')

In [28]:
ratings.shape

(1031130, 7)

### Ratings dataset should have ratings from users which exist in users dataset. Drop the remaining rows

In [29]:
ratings = pd.merge(ratings, users, on='userID', how='inner')

In [30]:
ratings.shape

(1031130, 9)

In [31]:
ratings.head(5)

,userID,ISBN,bookRating,bookTitle,bookAuthor,yearOfPublication,publisher,Location,Age
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,"tyler, texas, usa",34
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,"cincinnati, ohio, usa",23
2,2313,0812533550,9,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1986,Tor Books,"cincinnati, ohio, usa",23
3,2313,0679745580,8,In Cold Blood (Vintage International),TRUMAN CAPOTE,1994,Vintage,"cincinnati, ohio, usa",23
4,2313,0060173289,9,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,1996,HarperCollins,"cincinnati, ohio, usa",23


In [32]:
ratings = ratings[['userID','ISBN','bookRating']]

In [33]:
ratings.head()

,userID,ISBN,bookRating
0,276725,034545104X,0
1,2313,034545104X,5
2,2313,0812533550,9
3,2313,0679745580,8
4,2313,0060173289,9


### Consider only ratings from 1-10 and leave 0s in column `bookRating`

In [34]:
print(sorted(ratings['bookRating'].unique(),reverse=False))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [35]:
ratings = ratings.loc[ratings['bookRating']!=0]

In [36]:
print(sorted(ratings['bookRating'].unique(),reverse=False))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


### Find out which rating has been given highest number of times

In [37]:
import matplotlib.pyplot as plot
ratings.bookRating.value_counts().plot(kind='bar')

### **Collaborative Filtering Based Recommendation Systems**

### For more accurate results only consider users who have rated atleast 100 books

In [38]:
min_rating_users = ratings.groupby('userID').agg({'bookRating':[np.size]})
min_rating_users.head()

,bookRating
,size
userID,
8,7
9,1
12,1
14,3
16,1


In [43]:
top_users = min_rating_users['bookRating']['size']>=100

In [44]:
top_users.head()

userID
8     False
9     False
12    False
14    False
16    False
Name: size, dtype: bool

In [45]:
min_rating_users[top_users].sort_values([('bookRating','size')],ascending=False)[:15]

,bookRating
,size
userID,
11676,6943
98391,5689
189835,1899
153662,1845
23902,1180
235105,1020
76499,1012
171118,962


In [46]:
top_rated = pd.merge(ratings, min_rating_users[top_users], on='userID', how='inner')

C:\Users\sumans\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [47]:
top_rated.head(10)

,userID,ISBN,bookRating,"(bookRating, size)"
0,6543,0446605484,10,174
1,6543,0805062971,8,174
2,6543,0345342968,8,174
3,6543,0446610038,9,174
4,6543,0061009059,8,174
5,6543,0142001740,9,174
6,6543,0345436911,8,174
7,6543,038548951X,10,174
8,6543,0399149562,6,174
9,6543,0553574566,8,174


### Generating ratings matrix from explicit ratings


#### Note: since NaNs cannot be handled by training algorithms, replace these by 0, which indicates absence of ratings

In [48]:
top_rated = top_rated[['userID','ISBN','bookRating']]
top_rated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103269 entries, 0 to 103268
Data columns (total 3 columns):
userID        103269 non-null int64
ISBN          103269 non-null object
bookRating    103269 non-null int64
dtypes: int64(2), object(1)
memory usage: 3.2+ MB


In [49]:
top_rated.head()

,userID,ISBN,bookRating
0,6543,0446605484,10
1,6543,0805062971,8
2,6543,0345342968,8
3,6543,0446610038,9
4,6543,0061009059,8


In [50]:
ratings_pivoted = top_rated.pivot(index='userID',columns='ISBN',values='bookRating').fillna(0)
ratings_pivoted.iloc[:10, :10]

ISBN,0000913154,0001046438,000104687X,0001047213,0001047973,000104799X,0001048082,0001053736,0001053744,0001055607
userID,,,,,,,,,,
2033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Generate the predicted ratings using SVD with no.of singular values to be 50

In [51]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(top_rated[['userID', 'ISBN', 'bookRating']], reader)
trainset, testset = train_test_split(data, test_size=.15)
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True},n_factors=50)
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [52]:
test_pred = algo.test(testset)

### Take a particular user_id

### Lets find the recommendations for user with id `2110`

#### Note: Execute the below cells to get the variables loaded

In [53]:
userID = 2110
pred = []
for sublist in test_pred:
    if sublist[0] == userID:
        pred.append(sublist[3])
print(pred)        

[7.826277654993278, 7.826277654993278, 8.362637362637363, 7.826277654993278, 7.826277654993278, 8.362637362637363, 8.362637362637363, 8.362637362637363, 7.826277654993278, 8.362637362637363, 7.826277654993278, 8.362637362637363]


In [54]:
user_id = 2 #2nd row in ratings matrix and predicted matrix

### Get the predicted ratings for userID `2110` and sort them in descending order

In [91]:
pred.sort()
pred

[7.825411834400419,
 7.825411834400419,
 7.825411834400419,
 7.825411834400419,
 7.825411834400419,
 7.825411834400419,
 7.825411834400419,
 7.825411834400419,
 7.825411834400419,
 7.8896103896103895,
 8.318181818181818,
 8.318181818181818,
 8.318181818181818,
 8.318181818181818,
 8.318181818181818]

### Create a dataframe with name `user_data` containing userID `2110` explicitly interacted books

In [55]:
user_data = ratings.loc[ratings['userID']==2110]

In [56]:
user_data.head(10)

,userID,ISBN,bookRating
670337,2110,059035342X,10
670341,2110,0590448595,8
670344,2110,0451137965,9
670345,2110,0590629786,10
670346,2110,0590629794,10
670347,2110,0590629808,10
670348,2110,0345307674,10
670350,2110,0671001868,8
670352,2110,0515134384,5
670353,2110,0345314255,10


In [57]:
user_data.shape

(103, 3)

### Combine the user_data and and corresponding book data(`book_data`) in a single dataframe with name `user_full_info`

In [62]:
user_full_info = pd.merge(books, user_data, on='ISBN', how='inner')
user_full_info.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,userID,bookRating
0,0151008116,Life of Pi,Yann Martel,2002,Harcourt,2110,5
1,015216250X,So You Want to Be a Wizard: The First Book in ...,Diane Duane,2001,Magic Carpet Books,2110,8
2,0064472779,All-American Girl,Meg Cabot,2003,HarperTrophy,2110,8
3,0345307674,Return of the Jedi (Star Wars),James Kahn,1983,Del Rey Books,2110,10
4,0671527215,Hitchhikers's Guide to the Galaxy,Douglas Adams,1984,Pocket,2110,9


In [61]:
book_data.shape

NameError: name 'book_data' is not defined

In [60]:
book_data.head()

NameError: name 'book_data' is not defined

In [63]:
user_full_info.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,userID,bookRating
0,0151008116,Life of Pi,Yann Martel,2002,Harcourt,2110,5
1,015216250X,So You Want to Be a Wizard: The First Book in ...,Diane Duane,2001,Magic Carpet Books,2110,8
2,0064472779,All-American Girl,Meg Cabot,2003,HarperTrophy,2110,8
3,0345307674,Return of the Jedi (Star Wars),James Kahn,1983,Del Rey Books,2110,10
4,0671527215,Hitchhikers's Guide to the Galaxy,Douglas Adams,1984,Pocket,2110,9


### Get top 10 recommendations for above given userID from the books not already rated by that user

In [64]:
books_not_rated = pd.merge(user_full_info, books, on='ISBN', how='outer', indicator=True)

In [65]:
books_not_rated_by_2110 = books_not_rated.loc[books_not_rated._merge == 'right_only', ['ISBN']]

In [69]:
result = []
for index, row in books_not_rated_by_2110.iterrows(): 
    result.append(algo.predict(2110, row["ISBN"], verbose=False))

In [70]:
df_final = pd.DataFrame(result)

In [71]:
df_final.sort_values('est',ascending=False).iloc[:10,:10]

,uid,iid,r_ui,est,details
29943,2110,0740704818,None,10.0,"{'actual_k': 1, 'was_impossible': False}"
229071,2110,0394816269,None,10.0,"{'actual_k': 1, 'was_impossible': False}"
32170,2110,0804117942,None,10.0,"{'actual_k': 1, 'was_impossible': False}"
6332,2110,0451194861,None,10.0,"{'actual_k': 1, 'was_impossible': False}"
67834,2110,0004722124,None,10.0,"{'actual_k': 1, 'was_impossible': False}"
244754,2110,0762411236,None,10.0,"{'actual_k': 1, 'was_impossible': False}"
228897,2110,2831563798,None,10.0,"{'actual_k': 1, 'was_impossible': False}"
46537,2110,0425150984,None,10.0,"{'actual_k': 1, 'was_impossible': False}"
135440,2110,0671493191,None,10.0,"{'actual_k': 1, 'was_impossible': False}"
4788,2110,0061006629,None,10.0,"{'actual_k': 1, 'was_impossible': False}"
